# Stock AI Predictor - Parameter Testing on Google Colab

This notebook runs parameter testing for stock prediction patterns using Google Colab's high-performance environment. It's designed to handle computationally intensive parameter combinations, especially for short timeframes like 1-minute data.

## 1. Setup Environment

First, let's mount Google Drive and install required packages:

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository if not already in Drive
!git clone https://github.com/YOUR_USERNAME/Stock_AI_Predictor.git /content/Stock_AI_Predictor

# Or use existing repository from Drive
# !ln -s /content/drive/MyDrive/Stock_AI_Predictor /content/Stock_AI_Predictor

In [ ]:
# Install required packages
!pip install -r /content/Stock_AI_Predictor/requirements.txt
!pip install tqdm bcrypt sqlitecloud python-dotenv

## 2. Upload Database to Colab

### Option 1: Upload from local machine

In [ ]:
from google.colab import files
import os

# Create directory for database
!mkdir -p /content/Stock_AI_Predictor/Data/Storage

# Upload database file
uploaded = files.upload()
for filename in uploaded.keys():
    # Move uploaded file to the correct location
    !mv "{filename}" "/content/Stock_AI_Predictor/Data/Storage/data.db"
    print(f"Uploaded {filename} to /content/Stock_AI_Predictor/Data/Storage/data.db")

### Option 2: Copy from Google Drive

In [ ]:
import shutil

# Specify the source path in Google Drive
drive_db_path = '/content/drive/MyDrive/Stock_AI_Predictor/Data/Storage/data.db'

# Destination path in Colab
colab_db_path = '/content/Stock_AI_Predictor/Data/Storage/data.db'

# Ensure directory exists
os.makedirs(os.path.dirname(colab_db_path), exist_ok=True)

# Copy the file
if os.path.exists(drive_db_path):
    shutil.copy(drive_db_path, colab_db_path)
    print(f"Database copied from Drive to {colab_db_path}")
else:
    print(f"Database not found at {drive_db_path}")

### Option 3: Connect to SQLite Cloud (if available)

In [ ]:
# Create .env file with SQLite Cloud credentials
%%writefile /content/Stock_AI_Predictor/.env
SQLITECLOUD_URL="your_sqlitecloud_connection_string_here"

## 3. Import and Run Parameter Tester

In [ ]:
# Add the project root to sys.path
import sys
sys.path.append('/content/Stock_AI_Predictor')

# Import the Colab parameter tester
from Experements.ParamTesting.colab_parameter_tester import ColabParameterTester

# Create tester instance
db_path = '/content/Stock_AI_Predictor/Data/Storage/data.db'
tester = ColabParameterTester(db_path=db_path)

In [ ]:
# Check available stock and timeframe options
stocks = tester.list_stocks()
timeframes = tester.list_timeframes()

print("Available stocks:")
for stock_id, stock_name in stocks:
    print(f"  {stock_id}: {stock_name}")
    
print("\nAvailable timeframes:")
for tf_id, tf_name in timeframes:
    print(f"  {tf_id}: {tf_name}")

In [ ]:
# Run Parameter Tests - Choose One of the Options Below

## Option 1: Run for a specific stock and timeframe with date range
import datetime

# Set date range for testing (optional)
start_date = "2022-01-01"  # Can be None to use all available data
end_date = datetime.datetime.now().strftime("%Y-%m-%d")  # Use today as end date or None

# Run the test and collect the results
results, report = tester.run_parameter_testing_for_stock_timeframe_parallel(
    "GOLD", "M1", 
    start_date=start_date, 
    end_date=end_date
)

## Option 2: Run a quick test (comment out Option 1 if you want to use this)
# results = tester.run_quick_test("GOLD", "M1")

## Option 3: Compare hold period strategies (comment out Options 1 & 2 if you want to use this)
# results = tester.compare_hold_period_strategies("GOLD", "M1")

## Option 4: Run all tests for multiple timeframes (comment out Options 1-3 if you want to use this)
# all_results = tester.run_all_tests(stock_symbol="GOLD")

# Display top 5 parameter combinations if results exist
if results is not None and not isinstance(results, dict):
    print("\nTop 5 parameter combinations:")
    print(results.head(5))
elif results is not None and isinstance(results, dict):
    print("\nResults for multiple timeframes:")
    for tf, res in results.items():
        if not res.empty:
            print(f"\n{tf} - Top 3 combinations:")
            print(res.head(3))

# Check GPU status during execution
!nvidia-smi

## 4. Visualize Results

In [ ]:
# Generate and display visualization
if 'results' in locals() and results is not None and not isinstance(results, dict):
    # For single timeframe results
    fig = tester.visualize_results(results, "GOLD", "M1")
elif 'results' in locals() and results is not None and isinstance(results, dict):
    # For multiple timeframe results
    for tf, res in results.items():
        if not res.empty:
            print(f"\nVisualizing results for {tf}:")
            fig = tester.visualize_results(res, "GOLD", tf)
            
# If all_results exists (from Option 4)
elif 'all_results' in locals() and all_results is not None:
    for tf, res in all_results.items():
        if not res.empty:
            print(f"\nVisualizing results for {tf}:")
            fig = tester.visualize_results(res, "GOLD", tf)

In [ ]:
# Generate optimization report
if 'report' in locals() and report is not None:
    # If we already have a report from previous execution
    print(report)
elif 'results' in locals() and results is not None and not isinstance(results, dict):
    # For single timeframe results
    report = tester.generate_report(results, "GOLD", "M1")
    print(report)
elif 'results' in locals() and results is not None and isinstance(results, dict):
    # For multiple timeframe results
    for tf, res in results.items():
        if not res.empty:
            report = tester.generate_report(res, "GOLD", tf)
            print(f"\nReport for {tf}:")
            print(report)
            
# If all_results exists (from Option 4)
elif 'all_results' in locals() and all_results is not None:
    for tf, res in all_results.items():
        if not res.empty:
            report = tester.generate_report(res, "GOLD", tf)
            print(f"\nReport for {tf}:")
            print(report)

## 5. Save Results Back to Google Drive

In [ ]:
# Copy the updated database back to Google Drive
# This ensures your work is persisted
import shutil
import os

# Source path in Colab
colab_db_path = '/content/Stock_AI_Predictor/Data/Storage/data.db'

# Destination path in Google Drive
drive_db_dir = '/content/drive/MyDrive/Stock_AI_Predictor/Data/Storage'
os.makedirs(drive_db_dir, exist_ok=True)
drive_db_path = os.path.join(drive_db_dir, 'data.db')

# Copy the file
shutil.copy(colab_db_path, drive_db_path)
print(f"Updated database saved to {drive_db_path}")

## 6. Save Report and Visualizations

In [ ]:
# Save report to a file
import os

def save_report(report_content, timeframe):
    """Helper function to save a report to Google Drive"""
    if report_content:
        report_path = f'/content/drive/MyDrive/Stock_AI_Predictor/docs/parameter_optimization_report_{timeframe}.md'
        os.makedirs(os.path.dirname(report_path), exist_ok=True)
        with open(report_path, 'w') as f:
            f.write(report_content)
        print(f"Report saved to {report_path}")

# Save reports based on what data we have
if 'report' in locals() and report is not None:
    # If we have a single report
    save_report(report, "M1")  # Adjust timeframe as needed
    
elif 'results' in locals() and results is not None and isinstance(results, dict):
    # For multiple timeframe results
    for tf, res in results.items():
        if not res.empty:
            tf_report = tester.generate_report(res, "GOLD", tf)
            save_report(tf_report, tf)
            
# If all_results exists (from Option 4)
elif 'all_results' in locals() and all_results is not None:
    for tf, res in all_results.items():
        if not res.empty:
            tf_report = tester.generate_report(res, "GOLD", tf)
            save_report(tf_report, tf)

In [ ]:
# Save visualizations
import matplotlib.pyplot as plt
import os

def save_visualization(stock_symbol, timeframe):
    """Helper function to save a visualization to Google Drive"""
    vis_path = f'/content/drive/MyDrive/Stock_AI_Predictor/Images/ParamTesting/{stock_symbol}_{timeframe}_parameter_test.png'
    os.makedirs(os.path.dirname(vis_path), exist_ok=True)
    
    # Generate visualization again and save
    fig = tester.visualize_results(stock_symbol, timeframe, save_path=vis_path)
    print(f"Visualization saved to {vis_path}")
    return vis_path

# Save visualizations based on what data we have
if 'results' in locals() and results is not None and not isinstance(results, dict):
    # For single timeframe results
    save_visualization("GOLD", "M1")
    
elif 'results' in locals() and results is not None and isinstance(results, dict):
    # For multiple timeframe results
    for tf in results.keys():
        save_visualization("GOLD", tf)
        
# If all_results exists (from Option 4)
elif 'all_results' in locals() and all_results is not None:
    for tf in all_results.keys():
        save_visualization("GOLD", tf)

## 7. Download Database (Optional)

In [ ]:
# Download the updated database to your local machine
from google.colab import files
files.download('/content/Stock_AI_Predictor/Data/Storage/data.db')